In [ ]:
Generate a MSA for each candidate protein

In [ ]:
A. Generate a fasta database with all the candidate proteins
B. Run mmseqs on the fasta database vs Uniref90
C. Generate an MSA for each proteins
D. Realign with hhrealign

In [ ]:
import os 
from tqdm import tqdm

path_strains = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_session = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"

with open(f"{path_session}/all_prophage_proteins.db.fasta" ,"w") as outfile :
    for strain in tqdm(os.listdir(path_strains)):
        for prophage in os.listdir(f"{path_strains}/{strain}/tmp"):
            for protein in os.listdir(f"{path_strains}/{strain}/tmp/{prophage}"):
                sequence = open(f"{path_strains}/{strain}/tmp/{prophage}/{protein}").read()
                outfile.write(f"{sequence}\n")
                

In [ ]:
mmseqs convertalis \
/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session/input_db/all_prophage_proteins.db.mmseqs_input \
/home/conchae/databases/mmseqs_db/uniref90/uniref90_mmseqs \
/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session/mmseqs_out_t4 \
/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session/mmseqs_out_t4.m8 \
--db-load-mode 2 \
--format-output "query,target,qseq,tseq,evalue,bits,qaln,taln"



In [ ]:
import os 
import pandas as pd 
import subprocess
from multiprocessing import Pool
from tqdm import tqdm

path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"
path_strain = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
names_db = ["query","target","qseq","tseq","evalue","bits","qaln","taln"]
database_MSA = pd.read_csv(f"{path_db}/mmseqs_out_t4.m8", names = names_db, sep= "\t")

proteins = database_MSA["query"].unique()

def get_MSA(query) :
    path_strain = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
    strain, phage = query.split("__")[0] , query.split("__")[1]
    df_interest = database_MSA[database_MSA["query"] == query]
    dico_interest = df_interest.to_dict("records")
    with open(f"{path_strain}/{strain}/mmseqs_out/{phage}/{query}.MSA.fasta" ,"w") as outfile :
        for row in dico_interest :
            outfile.write(f">{row['target']}\n{row['tseq']}\n")
    with open(f"{path_db}/proteins_done","a+") as output :
        output.write(f"{query}\n")


if __name__ == '__main__':
    with Pool(40) as p:
        p.map(get_MSA, proteins)

In [ ]:
import os 
import pandas as pd 
import subprocess
from multiprocessing import Pool
from tqdm import tqdm


path_session = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"
path_strain = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"

df_names = pd.read_csv(f"{path_session}/all_prophage_proteins.names.db.fasta", names = ["protein"], header = None) 


def realign_MSA(query) :
    path_strain = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
    strain, phage = query.split("__")[0] , query.split("__")[1]
    file_query = f"{path_strain}/{strain}/tmp/{phage}/{query}"
    file_target =  f"{path_strain}/{strain}/mmseqs_out/{phage}/{query.split('.fasta')[0]}.MSA.fasta"
    if os.path.isfile(file_target)== True :
        # generate the multi fasta
        input_file =  f"{path_strain}/{strain}/mmseqs_out/{phage}/{query.split('.fasta')[0]}.multi.complete.fasta"
        make_input = f"cat {file_query} {file_target} > {input_file}"
        make_input_subprocess = subprocess.Popen (make_input , shell = True, stdout = subprocess.PIPE, stderr = subprocess.STDOUT)
        make_i_out, make_i_err = make_input_subprocess.communicate()
        print(make_i_out , input_file)
        # generate the MSA in a2m format
        file_out = f"{path_strain}/{strain}/mmseqs_out/{phage}/{query.split('.fasta')[0]}.MSA.a2m"
        command_realign = f"clustalo -i {input_file} -o {file_out} --outfmt=fa --threads=4"
        hhmer_subprocess = subprocess.Popen (command_realign , shell = True, stdout = subprocess.PIPE, stderr = subprocess.STDOUT)
        hmm_out, hmm_err = hhmer_subprocess.communicate()
        print(hmm_out ,hmm_err, "\n\n")
        with open(f"{path_session}/MSA_done","a+") as output :
            output.write(f"{query}\n")
    else :
         with open(f"{path_session}/MSA_missing","a+") as output :
            output.write(f"{query}\n")
        
if __name__ == '__main__':
    with Pool(20) as p:
        p.map(realign_MSA, df_names["protein"])